# Pre


In [ ]:
! pip install -U pandasql
import torch
import pandas as pd
import pandasql as ps
import numpy as np

In [2]:
from google.colab import drive 

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd drive/MyDrive/tarak/Players-Block-20211116T025030Z-001/Players-Block/GNN/data/

/content/drive/.shortcut-targets-by-id/1MeacoHD16_nfxfWKec4Nch65B3Nxp4K-/tarak/Players-Block-20211116T025030Z-001/Players-Block/GNN/data


In [5]:
!ls

 Competes_in_edges.csv	  gnn_data.pdf	        Plays_with_edges.csv
 Comp_season_nodes.csv	  Hosts_edges.csv      'Team_nodes(1).csv'
 Field_player_nodes.csv   Manager_nodes.csv     Team_nodes.csv
 GK_nodes.csv		  Manages_edges.csv     team_nodes_plus_features.csv
 gnn_data.docx		  Plays_for_edges.csv


In [9]:
PlaysWithEdgesDf = pd.read_csv('./Plays_with_edges.csv')
CompetesInEdgesDf = pd.read_csv('./Competes_in_edges.csv')
CompSeasonNodesDf = pd.read_csv ('Comp_season_nodes.csv')
FieldPlayerNodesDf = pd.read_csv('Field_player_nodes.csv')
GKNodesDf = pd.read_csv('GK_nodes.csv')
HostsEdgesDf = pd.read_csv('Hosts_edges.csv')
ManagerNodesDf = pd.read_csv('Manager_nodes.csv')
ManagesEdgesDf = pd.read_csv('Manages_edges.csv')
PlaysForEdgesDf = pd.read_csv('Plays_for_edges.csv')
TeamNodesDf = pd.read_csv('Team_nodes.csv')

# %cd ../../../../../../../../
# !ls
FeatureTeamNodesDf = pd.read_csv('team_nodes_plus_features.csv')

In [10]:
FeatureTeamNodesDf.columns

Index(['Unnamed: 0', 'teamName', 'postGame_side', 'competitionId', 'gameId',
       'teamId', 'preGame_steph', 'preGame_quality_overall',
       'preGame_quality_attack', 'preGame_quality_midfield',
       'preGame_quality_defence', 'preGame_proportion_pts', 'gameDate',
       'teamName.1', 'competitionName', 'nodeId', 'nodeName', 'year', 'month',
       'postGame_goals_scored', 'agg_postGame_goals_scored',
       'postGame_goals_conceded', 'agg_postGame_goals_conceded',
       'postGame_index', 'agg_postGame_index', 'postGame_total_shots',
       'agg_postGame_total_shots', 'postGame_total_xg',
       'agg_postGame_total_xg', 'postGame_total_big_chances',
       'agg_postGame_total_big_chances', 'postGame_total_key_passes',
       'agg_postGame_total_key_passes'],
      dtype='object')

In [11]:
FeatureTeamNodesDf = FeatureTeamNodesDf.fillna(0).reset_index()
FeatureTeamNodesDf.shape[0] // 2

4014

In [12]:
query = """ 
  select count(*)
  from FeatureTeamNodesDf T
  inner join HostsEdgesDf H
      on T.nodeName = H.dest_nodeName and T.gameId = H.gameId
"""
ps.sqldf(query)

,count(*)
0,4014


# Inference


In [13]:
HostsEdgesDf.columns

Index(['gameId', 'homeTeamId', 'awayTeamId', 'gameDate', 'seasonId',
       'competitionId', 'competitionName', 'homeTeamName', 'awayTeamName',
       'preGame_venueId', 'preGame_city', 'preGame_country', 'preGame_lat',
       'preGame_long', 'preGame_field_length_m', 'preGame_field_width_m',
       'preGame_capacity', 'preGame_surface', 'preGame_is_behind_closed_doors',
       'preGame_homeTeamDominance', 'preGame_awayTeamDominance',
       'preGame_venueName', 'postGame_tgt_half_gd', 'postGame_tgt_gd',
       'postGame_tgt_homeTeamGoals', 'postGame_tgt_awayTeamGoals',
       'postGame_tgt_half_homeTeamGoals', 'postGame_tgt_half_awayTeamGoals',
       'postGame_tgt_outcome', 'postGame_tgt_half_outcome', 'preGame_odds1',
       'preGame_odds2', 'preGame_oddsX', 'src_nodeId', 'src_nodeName',
       'edge_type', 'dest_nodeId', 'dest_nodeName'],
      dtype='object')

In [14]:
query = """ 
  select preGame_steph, preGame_quality_overall,
          preGame_quality_attack, preGame_quality_midfield,
          preGame_quality_defence, preGame_proportion_pts,
          agg_postGame_goals_scored,
          agg_postGame_goals_conceded,
          agg_postGame_index,
          agg_postGame_total_shots,
          agg_postGame_total_xg,
          agg_postGame_total_big_chances,
          agg_postGame_total_key_passes,
          preGame_odds1,
          preGame_odds2,
          preGame_oddsX,
          preGame_homeTeamDominance,
          preGame_awayTeamDominance
  from FeatureTeamNodesDf T inner join HostsEdgesDf H
    on T.nodeName = H.dest_nodeName
  order by T.gameDate, T.gameId
"""
away_features = ps.sqldf(query)
query = """ 
  select preGame_steph, preGame_quality_overall,
          preGame_quality_attack, preGame_quality_midfield,
          preGame_quality_defence, preGame_proportion_pts,
          agg_postGame_goals_scored,
          agg_postGame_goals_conceded,
          agg_postGame_index,
          agg_postGame_total_shots,
          agg_postGame_total_xg,
          agg_postGame_total_big_chances,
          agg_postGame_total_key_passes,
          preGame_odds1,
          preGame_odds2,
          preGame_oddsX,
          preGame_homeTeamDominance,
          preGame_awayTeamDominance
  from FeatureTeamNodesDf T inner join HostsEdgesDf H
    on T.nodeName = H.src_nodeName
  order by T.gameDate, T.gameId
"""
home_features = ps.sqldf(query)
ps.sqldf(query)

query = """
  select postGame_tgt_gd
  from HostsEdgesDf H inner join FeatureTeamNodesDf T
    on H.src_nodeName = T.nodeName
  order by T.gameDate, T.gameId
"""
results = ps.sqldf(query)

In [15]:
results = torch.from_numpy(results.values).view(-1)
dummy = 6
results[results > 0] = dummy + 2
results[results == 0] = dummy + 1
results[results < 0] = dummy + 0
results -= dummy

home_features = torch.from_numpy(home_features.values)
away_features = torch.from_numpy(away_features.values)

# Blade Chest Model

In [16]:
from torch.nn import Module, Linear, NLLLoss, Tanh, BatchNorm1d, Dropout, LogSoftmax, PairwiseDistance
from torch.optim import Adam
from torchsummary import summary

In [31]:
class BladeChest(Module):
  def __init__(self, feature_size, hidden_size, dropout_rate=0.5):
    super(BladeChest, self).__init__()
    self.feature_size = feature_size
    self.hidden_size = hidden_size
    self.blade_transfrom = Linear(self.feature_size, self.hidden_size)
    self.chest_transform = Linear(self.feature_size, self.hidden_size)
    self.activation = Tanh()
    self.feature_bn = BatchNorm1d(self.feature_size)
    self.hidden_bn = BatchNorm1d(self.hidden_size)
    self.regularizer = Dropout(dropout_rate)
    self.result_transform = Linear(1, 3)
    self.distance_fn = PairwiseDistance()
    self.classifier = LogSoftmax(-1)

  def _matchup_score(self, away_blade, home_chest, home_blade, away_chest):
    return self.distance_fn(away_blade, home_chest).pow(2) - self.distance_fn(home_chest, away_blade).pow(2)

  def encode_team(self, x_team):
    h_team = self.feature_bn(x_team)

    team_blade = self.blade_transfrom(h_team)
    team_blade = self.hidden_bn(team_blade)
    team_blade = self.regularizer(team_blade)

    team_chest = self.chest_transform(h_team)
    team_chest = self.hidden_bn(team_chest)
    team_chest = self.regularizer(team_chest)

    return team_blade, team_chest

  def forward(self, x_home, x_away):
    home_blade, home_chest = self.encode_team(x_home)
    away_blade, away_chest = self.encode_team(x_away)
    h_home = self.feature_bn(x_home)
    h_away = self.feature_bn(x_away)

    matchup = self._matchup_score(away_blade, home_chest, home_blade, away_chest).view(-1, 1)
    result_h = self.result_transform(matchup)
    result_h = self.regularizer(result_h)

    return self.classifier(result_h)

In [19]:
results.shape

torch.Size([4014])

In [20]:
train_size = 3200
validation_size = 400
batch_size = 32
rand_indcs = torch.randperm(results.shape[0])
train_indcs = rand_indcs[:train_size]
val_indcs = rand_indcs[train_size: train_size + validation_size]
test_indcs = rand_indcs[train_size + validation_size:]
train_home_features = home_features[train_indcs, :].reshape(-1, batch_size, away_features.shape[-1])
train_away_features = away_features[train_indcs, :].reshape(-1, batch_size, away_features.shape[-1])
train_results = results[train_indcs].reshape(-1, batch_size)

val_home_features = home_features[val_indcs, :]
val_away_features = away_features[val_indcs, :]
val_results = results[val_indcs]
test_home_features = home_features[test_indcs, :]
test_away_features = away_features[test_indcs, :]
test_results = results[test_indcs]

In [21]:
def train(model, home, away, results, optimizer, criterion):
  model.train()
  optimizer.zero_grad()
  output = model(home, away)
  loss = criterion(output, results)
  loss.backward()
  optimizer.step() 
  return loss.item()


@torch.no_grad()
def eval(model, home, away, results):
  model.eval()
  output = model(home, away).argmax(-1)
  correct = (output == results).sum().item()
  return (correct / results.shape[0])

In [28]:
feature_size = home_features.shape[-1]
bc_model = BladeChest(feature_size, 128, 0.4).double()
######################################
scaled_class_weight = torch.tensor([1.5, 1.8, 1], dtype=torch.double)
#####################################
criterion = NLLLoss(scaled_class_weight)
print(f'Initial Random State Model: ')
print(f'Validation Acc: {eval(bc_model, val_home_features, val_away_features, val_results) * 100: .4f}%')
print(f'Test Acc: {eval(bc_model, test_home_features, test_away_features, test_results) * 100: .4f}%')

Initial Random State Model: 
Validation Acc:  24.5000%
Test Acc:  25.1208%


In [29]:
optimizer = Adam(bc_model.parameters(), lr=1e-3, weight_decay=0)
try:
  for epoch in range(1500000):
    b_loss = 0
    for home, away, result in zip(train_home_features, train_away_features, train_results):
      b_loss += train(bc_model, home, away, result, optimizer, criterion)
    print(f'Epoch: {epoch + 1}')
    print(f'Batch Avg. Loss: {b_loss / batch_size: .4f}')
    print(f'Train Acc: {eval(bc_model, train_home_features.reshape(-1, feature_size), train_away_features.reshape(-1, feature_size), train_results.reshape(-1)) * 100: .4f}%')
    print(f'Validation Acc: {eval(bc_model, val_home_features, val_away_features, val_results) * 100: .4f}%')
    print('=' * 64)
except KeyboardInterrupt:
  print(f'Test Acc: {eval(bc_model, test_home_features, test_away_features, test_results) * 100: .4f}%')

Epoch: 1
Batch Avg. Loss:  4.4175
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 2
Batch Avg. Loss:  4.2273
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 3
Batch Avg. Loss:  4.0559
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 4
Batch Avg. Loss:  3.9364
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 5
Batch Avg. Loss:  3.8019
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 6
Batch Avg. Loss:  3.6854
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 7
Batch Avg. Loss:  3.6327
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 8
Batch Avg. Loss:  3.5759
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 9
Batch Avg. Loss:  3.5206
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 10
Batch Avg. Loss:  3.4773
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 11
Batch Avg. Loss:  3.4715
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 12
Batch Avg. Loss:  3.4454
Train Acc:  25.8438%
Validation Acc:  24.5000%
Epoch: 13
Batch Avg. Loss

In [30]:
bc_model.eval()
out = bc_model(train_home_features.reshape(-1, feature_size), train_away_features.reshape(-1, feature_size))
(out.argmax(dim=1) == 1).sum()

tensor(580)